<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#计算图的使用" data-toc-modified-id="计算图的使用-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>计算图的使用</a></span><ul class="toc-item"><li><span><a href="#使用默认的计算图" data-toc-modified-id="使用默认的计算图-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>使用默认的计算图</a></span></li><li><span><a href="#产生新的计算图" data-toc-modified-id="产生新的计算图-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>产生新的计算图</a></span></li><li><span><a href="#在计算图中指定计算设备" data-toc-modified-id="在计算图中指定计算设备-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>在计算图中指定计算设备</a></span></li></ul></li></ul></div>

# 计算图的使用

## 使用默认的计算图

TensorFlow的程序一般分为两个阶段：**构建计算图**和**执行计算图**。
在Tensorflow程序中，系统会自动维护一个默认的计算图，通过`tf.get_default_graph`函数可以获取默认的计算图

In [1]:
import tensorflow as tf
import numpy as np

a = tf.constant([1.0, 2.0], name='a')
b = tf.constant([2.0, 3.0], name='b')
result = tf.add(a, b)

# 获取张量a所属的计算图和系统默认的计算图
# 它们两者应该时相等的
print("a graph is: ", a.graph)
print("the default graph is: ", tf.get_default_graph())

a graph is:  <tensorflow.python.framework.ops.Graph object at 0x000001CCAC562DD8>
the default graph is:  <tensorflow.python.framework.ops.Graph object at 0x000001CCAC562DD8>


## 产生新的计算图

除了使用默认的计算图以外，TensorFlow支持通过`tf.Graph`函数来生成新的计算图。
不同计算图上的张量和运算都不会共享。以下代码示意了在不同计算图上定义和使用变量。

In [1]:
import tensorflow as tf

g1 = tf.Graph()
with g1.as_default():
    # 在计算图g1上定义变量"v"，并设置值为0
    v = tf.get_variable("v", shape=[1], initializer=tf.zeros_initializer())
    
g2 = tf.Graph()
with g2.as_default():
    # 在计算图g2上定义变量"v"，并设置值为1
    v = tf.get_variable("v", shape=[1], initializer=tf.ones_initializer())
    
# 在计算图g1中读取变量"v"的取值
with tf.Session(graph=g1) as sess:
    # 变量初始化
    tf.global_variables_initializer().run()
    
    with tf.variable_scope("", reuse=True):
        # 在计算图g1中，变量"v"的取值应该是0，所以下面的结果应该是[0.]
        print(sess.run(tf.get_variable("v")))
        
# 在计算图g2中读取变量"v"的取值
with tf.Session(graph=g2) as sess:
    # 变量初始化
    tf.global_variables_initializer().run()
    
    with tf.variable_scope("", reuse=True):
        # 在计算图g1中，变量"v"的取值应该是0，所以下面的结果应该是[0.]
        print(sess.run(tf.get_variable("v")))


[0.]
[1.]


上面的代码产生了两个计算图，每个计算图中定义了一个名字为 `v` 的变量。在计算图中 `g1` 中，将 `v` 初始化为 $0$ ；在计算图 `g2` 中，将 `v` 初始化为 $1$。可以看到当运行不同的计算图时，`v` 的变量值也不一样。

## 在计算图中指定计算设备

TensorFlow中的计算图不仅仅可以用来隔离张量和计算，它还提供了管理张量和计算的功能。计算图可以用`tf.Graph.device` 函数来指定运行计算的设备。这是 TensorFlow 使用GPU提供了机制。以下程序可以让加法跑在GPU上。

In [3]:
import tensorflow as tf

with tf.Graph().as_default() as g:
    with g.device("/gpu:0"):
        a = tf.get_variable("a", shape=[2], initializer=tf.zeros_initializer())
        b = tf.get_variable("b", shape=[2], initializer=tf.ones_initializer())
        result = tf.add(a, b)

# 配置输出参数：打印设备，运行自动计算设备
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
with tf.Session(config=config,graph=g) as sess:
    # 变量初始化操作
    tf.global_variables_initializer().run()
    
    with tf.variable_scope("", reuse=True):
        # 在计算图g中，获取变量result
        print(sess.run(result))

[1. 1.]
